<h1><center>Building LSTM Model</center></h1>

### Installing required lib

In [1]:
# !pip install keras
# !pip install tensorflow

#### Importing required lib

In [2]:
import numpy as np # linefinalar algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
# from keras.utils.np_utils import to_categorical
import re

#Models
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

import torch

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [4]:
# reading sample file

data = pd.read_excel('bert_sample.xlsx')

### Descriptive statistics

In [5]:
print(data.shape)

(10000, 2)


In [6]:
data.head(5)

,ITEM_NAME,CATEGORY_ID
0,CALIBRACION TRANSDUCER 75 nm,CAPITAL ASSEMBLY
1,for pusher whskey,CAPITAL ASSEMBLY
2,Stat 40B Press Head Cup to Carrier from Stati...,CAPITAL ASSEMBLY
3,TRANSD. Cable (4145097103) scrw,CAPITAL ASSEMBLY
4,"ZT200 7,5BAR,13BAR60HZ NUMERO DE SERIE: AIF09...",CAPITAL ASSEMBLY


# Text pre-processing

In [7]:
#Converting allthe text to lower case and apply regular expression to remove unwanted characters
data['ITEM_NAME'] = data['ITEM_NAME'].apply(lambda x: x.lower())
data['ITEM_NAME'] = data['ITEM_NAME'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [8]:
data.head(5)

,ITEM_NAME,CATEGORY_ID
0,calibracion transducer 75 nm,CAPITAL ASSEMBLY
1,for pusher whskey,CAPITAL ASSEMBLY
2,stat 40b press head cup to carrier from stati...,CAPITAL ASSEMBLY
3,transd cable 4145097103 scrw,CAPITAL ASSEMBLY
4,zt200 75bar13bar60hz numero de serie aif09446...,CAPITAL ASSEMBLY


In [9]:
#Tokenization of words which converts word format to numerical format to feed into algorithms

max_fatures = 2000
tokenizer = Tokenizer(nb_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['ITEM_NAME'].values)
X = tokenizer.texts_to_sequences(data['ITEM_NAME'].values)
X = pad_sequences(X,maxlen=100)

C:\Users\hanitha\anaconda3\lib\site-packages\keras_preprocessing\text.py:180: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [10]:
X.shape

(10000, 100)

### Analysing target variable

In [11]:
data.CATEGORY_ID.value_counts()

CUTTING TOOLS        5000
CAPITAL ASSEMBLY     2000
CHEMICALS            2000
LOGISTICS SERVICE    1000
Name: CATEGORY_ID, dtype: int64

In [12]:
#creating fuction to make target variable into numerical format

def Getclassification(CATEGORY_ID):
    if CATEGORY_ID =="CAPITAL ASSEMBLY":
        category = 1
    elif CATEGORY_ID =="CHEMICALS":
            category = 2   
    elif CATEGORY_ID == 'CUTTING TOOLS':
            category = 3
    else:
            category = 4

    return category

In [13]:
data['categoryNum'] = data.CATEGORY_ID.apply(Getclassification)

In [14]:
data.head(5)

,ITEM_NAME,CATEGORY_ID,categoryNum
0,calibracion transducer 75 nm,CAPITAL ASSEMBLY,1
1,for pusher whskey,CAPITAL ASSEMBLY,1
2,stat 40b press head cup to carrier from stati...,CAPITAL ASSEMBLY,1
3,transd cable 4145097103 scrw,CAPITAL ASSEMBLY,1
4,zt200 75bar13bar60hz numero de serie aif09446...,CAPITAL ASSEMBLY,1


In [15]:
data.categoryNum.value_counts(), data.CATEGORY_ID.value_counts()

(3    5000
 2    2000
 1    2000
 4    1000
 Name: categoryNum, dtype: int64,
 CUTTING TOOLS        5000
 CAPITAL ASSEMBLY     2000
 CHEMICALS            2000
 LOGISTICS SERVICE    1000
 Name: CATEGORY_ID, dtype: int64)

# LSTM Model building

In [29]:
#Creating LSTM model object

embed_dim = 256
lstm_out = 196

LSTMmodel = Sequential()
LSTMmodel.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
LSTMmodel.add(LSTM(lstm_out))
LSTMmodel.add(Dense(4,activation='softmax'))
LSTMmodel.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(LSTMmodel.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 256)          512000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               355152    
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 788       
Total params: 867,940
Trainable params: 867,940
Non-trainable params: 0
_________________________________________________________________
None


## Train test split

In [30]:
#Dummification of target variable
Y = pd.get_dummies(data['categoryNum']).values

### Train test split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(6700, 100) (6700, 4)
(3300, 100) (3300, 4)


In [31]:
X_train

array([[   0,    0,    0, ...,    0,  551,  135],
       [   0,    0,    0, ...,    5,   71, 1242],
       [   0,    0,    0, ...,    1,  197,    1],
       ...,
       [   0,    0,    0, ...,    0,  285,   52],
       [   0,    0,    0, ..., 1079,  552,  911],
       [   0,    0,    0, ...,  132,  209,   69]])

In [32]:
Y_train[1]

array([0, 0, 1, 0], dtype=uint8)

### Training of model

In [33]:
%%time

batch_size = 64

LSTMmodel.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 2)

Epoch 1/10
105/105 - 15s - loss: 0.8560 - accuracy: 0.6531
Epoch 2/10
105/105 - 15s - loss: 0.4146 - accuracy: 0.8490
Epoch 3/10
105/105 - 14s - loss: 0.3150 - accuracy: 0.8821
Epoch 4/10
105/105 - 15s - loss: 0.2693 - accuracy: 0.8973
Epoch 5/10
105/105 - 14s - loss: 0.2434 - accuracy: 0.9067
Epoch 6/10
105/105 - 14s - loss: 0.2261 - accuracy: 0.9128
Epoch 7/10
105/105 - 14s - loss: 0.2241 - accuracy: 0.9145
Epoch 8/10
105/105 - 15s - loss: 0.2080 - accuracy: 0.9185
Epoch 9/10
105/105 - 15s - loss: 0.1932 - accuracy: 0.9237
Epoch 10/10
105/105 - 15s - loss: 0.1872 - accuracy: 0.9257
Wall time: 2min 26s


In [34]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]

In [36]:
X_test.shape

(1800, 100)

In [37]:
# validation_size = 1500

# X_validate = X_test[-validation_size:]
# Y_validate = Y_test[-validation_size:]
# X_test = X_test[:-validation_size]
# Y_test = Y_test[:-validation_size]
score,acc = LSTMmodel.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

29/29 - 2s - loss: 0.6766 - accuracy: 0.8294
score: 0.68
acc: 0.83


#### Getting class prediction with percentage

In [38]:
capital_cnt, chemical_cnt, capital_correct, chemical_correct, cuttingTools_cnt, cuttingTools_correct, LogSev_correct, LogSer_cnt = 0,0,0,0,0,0,0,0
for x in range(len(X_validate)):    
    result = LSTMmodel.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 1:
            capital_correct += 1
        elif np.argmax(Y_validate[x]) == 2:
            chemical_correct += 1
        elif np.argmax(Y_validate[x]) == 3:
            cuttingTools_correct += 1
        else:
            LogSev_correct += 1
       
    if np.argmax(Y_validate[x]) == 1:
        capital_cnt += 1
    elif np.argmax(Y_validate[x]) == 2:
        chemical_cnt += 1
    elif np.argmax(Y_validate[x]) == 3:
        cuttingTools_cnt += 1
    else:
        LogSer_cnt += 1

1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1

1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1

In [40]:
print('capital_accuracy', round(capital_correct/capital_cnt*100, 2), '%')
print ('chemical_accuracy', round(chemical_correct/chemical_cnt*100,2), "%")
print('cuttingTools_accuracy', round(cuttingTools_correct/cuttingTools_cnt*100, 2), '%')
print('LogSev_accuracy', round(LogSev_correct/LogSer_cnt*100, 2), '%')

capital_accuracy 69.97 %
chemical_accuracy 95.21 %
cuttingTools_accuracy 66.89 %
LogSev_accuracy 72.64 %


In [56]:
print("Capital:", capital_correct,'/',capital_cnt)
print('Chemical:', chemical_correct,'/', chemical_cnt)
print('CuttingTools:', cuttingTools_correct,'/',cuttingTools_cnt)
print('LogisticService', LogSev_correct,'/',LogSer_cnt)
print('***********************************')
print('\n')
print('capital_accuracy:', round(capital_correct/capital_cnt*100, 2), '%')
print ('chemical_accuracy:', round(chemical_correct/chemical_cnt*100,2), "%")
print('cuttingTools_accuracy:', round(cuttingTools_correct/cuttingTools_cnt*100, 2), '%')
print('LogSev_accuracy:', round(LogSev_correct/LogSer_cnt*100, 2), '%')

Capital: 205 / 293
Chemical: 716 / 752
CuttingTools: 99 / 148
LogisticService 223 / 307
***********************************


capital_accuracy: 69.97 %
chemical_accuracy: 95.21 %
cuttingTools_accuracy: 66.89 %
LogSev_accuracy: 72.64 %


**LSTM model has good accuracy for the 'Chemical' class than other**